## Using (most of) the chip to encode the 3 qubit triangle

In [1]:
# Handle the relevant import statements before moving on...
import sys
sys.path.append("../dwaveutils/dwavetools")
sys.path.append("../dwaveutils/probrep")
from dictrep import DictRep
from dwave.system.samplers import DWaveSampler
from dwavetools import (find_heff_s, make_dwave_schedule, create_heff_csv,
                        nqubit_1pauli, nqubit_2pauli, loadAandB,
                        make_numeric_schedule, time_interpolation,
                        get_numeric_H, densify_unitcell)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qutip as qt
from operator import add

Load in chip data and get working qubits/couplers. 
Also, "block_H" refers to the specific qubits that connect the "blocks" that make up the massive "1 qubit chains." 

In [2]:
chip = DWaveSampler()
wqubits = chip.properties['qubits']
wcouplers = chip.properties['couplers']
block_H = {(44, 52): 'J2', (1243, 1371): 'J1', (1168, 1296): 'J1'}

Here, we will form the large "blocks" that densley connect a bunch of qubits in a rectangle that represents a single qubit.

In [3]:
def dense_rectangle_qubit(fqbit, width, height, hweight, Jweight):
    '''
    Creates a width x height densely connected qubit chain starting with fqbit, giving
    each qubit along the chain a linear weight of hweight and coupling strength of Jweight. 
    '''
    q_H = {}
    
    ucell = 1
    while ucell <= (width*height):
        print(ucell)
        print(fqbit)
        # if not at the edge of rectangle, go to next unit cell to the right
        # and make couplings that connect these unit cells
        if ucell % width != 0: 
            couplings = [[fqbit+4+i, fqbit+12+i] for i in range(4)]
            fqbit += 8
        # otherwise, go to next row and connect edge unit cell with those BELOW
        # unless you are actually on the final row, of course; then don't connect down
        elif (ucell % width == 0) and (ucell < width*height):
            couplings = [[fqbit+i, fqbit+128+i] for i in range(4)]
            fqbit += 88

        print("Right before this function call.")
        uc_H = densify_unitcell(fqbit, hweight, Jweight, wqubits, wcouplers)
        print("Right after this function call.")
        q_H.update(uc_H)
        count = 0
        for coupling in couplings:
            # print("I'm in this loop.")
            if coupling in wcouplers:
                q_H.update({tuple(coupling): Jweight})
                count += 1         
        if count == 0:
            return("Chain is broken; no connections availible.")
            break

        ucell += 1
        
    return q_H

In [4]:
# create the first qubit by starting with qubit 0 and creating a dense 6 x 10 block
first_qubit = dense_rectangle_qubit(0, 6, 10, 1, 1)

1
0
Right before this function call.
8
12
9
13
10
14
11
Right after this function call.
2
8
Right before this function call.
16
20
17
21
18
22
19
Right after this function call.
3
16
Right before this function call.
24
28
25
29
26
30
27
Right after this function call.
4
24
Right before this function call.
32
36
33
37
34
38
35
Right after this function call.
5
32
Right before this function call.
40
44
41
45
42
46
43
Right after this function call.
6
40
Right before this function call.
128
132
129
133
130
134
131
Right after this function call.
7
128
Right before this function call.
136
140
137
141
138
142
139
Right after this function call.
8
136
Right before this function call.
144
148
145
149
146
150
147
Right after this function call.
9
144
Right before this function call.
152
156
153
157
154
158
155
Right after this function call.
10
152
Right before this function call.
160
164
161
165
162
166
163
Right after this function call.
11
160
Right before this function call.
168
172
169
17

In [5]:
# create the second qubit by starting with qubit 48 and creating another dense 6 x 10 block
second_qubit = dense_rectangle_qubit(48, 6, 10, 1, 1)

1
48
Right before this function call.
56
60
57
61
58
62
59
Right after this function call.
2
56
Right before this function call.
64
68
65
69
66
70
67
Right after this function call.
3
64
Right before this function call.
72
76
73
77
74
78
75
Right after this function call.
4
72
Right before this function call.
80
84
81
85
82
86
83
Right after this function call.
5
80
Right before this function call.
88
92
89
93
90
94
91
Right after this function call.
6
88
Right before this function call.
176
180
177
181
178
182
179
Right after this function call.
7
176
Right before this function call.
184
188
185
189
186
190
187
Right after this function call.
8
184
Right before this function call.
192
196
193
197
194
198
195
Right after this function call.
9
192
Right before this function call.
200
204
201
205
202
206
203
Right after this function call.
10
200
Right before this function call.
208
212
209
213
210
214
211
Right after this function call.
11
208
Right before this function call.
216
220
21

KeyboardInterrupt: 

In [57]:
# create the final qubit by starting with qubit 1280 and creating a dense 10 x 6 block
#third_qubit = dense_rectangle_qubit(1280, 10, 6, 1, 1)